In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5"

import torch_influence
import torchvision
import torch
import numpy as np
import matplotlib.pyplot as plt
torch.set_warn_always(False)

from typing import List
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import yaml
import lm_eval

import datasets
import os
import sys
from typing import List

import fire
import torch
import transformers
from datasets import load_dataset

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training ,
    set_peft_model_state_dict,
)
from transformers import LlamaForCausalLM, LlamaTokenizer

%load_ext autoreload
%autoreload 2

In [ ]:

# dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", cache_dir = "./datasets") # wikitext; evaluation broken DONE
# print("wikitext")
# print(dataset)

# dataset = datasets.load_dataset("trivia_qa", "rc", cache_dir = "./datasets") # triviaQA DONE
# print("trivia_qa")
# print(dataset)

# dataset = datasets.load_dataset("bigbio/pubmed_qa", cache_dir = "./datasets") # DONE
# print("pubmed_qa")
# print(dataset)

dataset = datasets.load_dataset("truthfulqa/truthful_qa", "generation", cache_dir = "./datasets") # truthful QA # DONE
print("truthful_qa")
print(dataset)

dataset = datasets.load_dataset("tau/commonsense_qa", cache_dir = "./datasets") # commonsense DONE
print("commonsense_qa")
print(dataset)

dataset = datasets.load_dataset("DatologyAI/hellaswag", cache_dir = "./datasets", trust_remote_code=True) # hellaswag
print("hellaswag")
print(dataset)

# dataset = datasets.load_dataset("allenai/sciq", cache_dir = "./datasets")
# print("sciq")
# print(dataset)

dataset = datasets.load_dataset("openai/gsm8k", "main")
print("gsm8k")
print(dataset)

# dataset = datasets.load_dataset("rajpurkar/squad_v2" , cache_dir = "./datasets") # DONE
# print("squad_v2")
# print(dataset)

# dataset = datasets.load_dataset("dvilares/head_qa", cache_dir = "./datasets", trust_remote_code=True) # health QA
# print("head_qa")
# print(dataset)

# ['wikitext-103-raw-v1', 'wikitext-103-v1', 'wikitext-2-raw-v1', 'wikitext-2-v1']

In [12]:
# dataset = datasets.load_dataset("truthfulqa/truthful_qa", "generation", cache_dir = "./datasets") # truthful QA # DONE
# print("truthful_qa")
# print(dataset)

# dataset = datasets.load_dataset("tau/commonsense_qa", cache_dir = "./datasets") # commonsense DONE
# print("commonsense_qa")
# print(dataset)

# dataset = datasets.load_dataset("DatologyAI/hellaswag", cache_dir = "./datasets", trust_remote_code=True) # hellaswag
# print("hellaswag")
# print(dataset)

dataset = datasets.load_dataset("openai/gsm8k", "main")
print("gsm8k")
print(dataset)

gsm8k
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})


In [16]:
dataset["train"][0:5]['question']

['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?',
 'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?',
 'Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?',
 'James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?']

In [6]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

In [7]:
model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-1.5B-instruct", trust_remote_code=True, device="cpu")
model = model.to("cuda:4")


2025-02-06:16:39:37,828 INFO     [SentenceTransformer.py:218] Load pretrained SentenceTransformer: Alibaba-NLP/gte-Qwen2-1.5B-instruct
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.80it/s]
2025-02-06:16:39:43,668 INFO     [SentenceTransformer.py:357] 1 prompts are loaded, with the keys: ['query']


In [ ]:
from llm import load_data
import time
import numpy as np
from sklearn.preprocessing import normalize
task_metrics = {
  #"commonsense_qa": "acc,none",
  "gsm8k": "exact_match,strict-match",
  #"headqa_en": "acc,none",
  #"hellaswag": "acc,none",
  #"pubmedqa": "acc,none",
  #"sciq": "acc_norm,none",
  #"triviaqa": "exact_match,remove_whitespace",
  #"truthfulqa_gen": "bleu_acc,none",
  #"wikitext": "word_perplexity,none",
}

def embed(example):
  try:
    max_length = 32768
    start = time.time()
    text = example["text"]
    # if len(text) > 400:
    #   text = text[:200]
    passage_embeddings = model.encode([text])
    # normalize embeddings
    query_embeddings = normalize(passage_embeddings)
  except:
    assert False
  return {"embeddings": query_embeddings}

def extract_string(example):
  return {"text": example["question"]}
  
for task in task_metrics.keys():
    train_dataset, val_dataset = load_data(data_domain=task)
    
    string_dataset_train = train_dataset.map(extract_string)
    embed_dataset_train = string_dataset_train.map(embed)
    output_embedding = np.array(embed_dataset_train["embeddings"]).astype(str)
    np.save("embeddings" +str(task) + ".npy", output_embedding)

# LLM/domain_training_embeddings/commonsense_qa.npy

In [ ]:
batch_size = 16
num_epochs = 100
learning_rate= 3e-4
cutoff_len = 256
val_set_size = 2000
# lora hyperparams
lora_r = 8
lora_alpha = 16
lora_dropout= 0.05
lora_target_modules = [
    "q_proj",
    "v_proj",
]
# llm hyperparams
train_on_inputs = False  # if False, masks out inputs in loss
add_eos_token = False
group_by_length = False  # faster, but produces an odd training loss curve
gradient_accumulation_steps = 1
output_dir = "LLM/"
use_wandb = False

# Load the TrivialQA dataset
dataset = datasets.load_dataset("trivia_qa", "rc", cache_dir = "./datasets")

train_dataset = dataset["train"]
val_dataset = dataset["validation"]


# get tokenizer and model
tokenizer, model = get_tokenizer_and_model(model_id = "LLM/llama_8b_instruct")
#model = prepare_model_for_kbit_training(model)

# download data
indices = range(0,1000)
train_dataset = dataset["train"].select(indices)
val_dataset = dataset["validation"].select(indices)

config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)


model.print_trainable_parameters()  # Be more transparent about the % of trainable params.

# apply tokenization to the data
print("tokenizing data into correct format...")
train_data = (
        train_dataset.shuffle().map(generate_and_tokenize_prompt_trivialQA)
    )

val_data = (
        val_dataset.shuffle().map(generate_and_tokenize_prompt_trivialQA)
    )

if torch.cuda.device_count() > 1:
    # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
    model.is_parallelizable = True
    model.model_parallel = True

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_eval_batch_size=batch_size,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        num_train_epochs=5,
        learning_rate=learning_rate,
        bf16=True,
        logging_steps=10,
        optim="adamw_torch",
        evaluation_strategy="steps" if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=100 if val_set_size > 0 else None,
        save_steps=200,
        output_dir=output_dir,
        save_total_limit=3,
        load_best_model_at_end=True if val_set_size > 0 else False,
        ddp_find_unused_parameters=True,
        group_by_length=group_by_length,
        report_to="wandb"
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)
print("begin to train...")
trainer.train()
